# 기본데이터 만들기 (처음 1번만)

In [165]:
# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp
# ======================================================================================================================

scaler = MinMaxScaler()
random.seed(54654)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

label = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_label.csv')



# ======================================================================================================================
print("process - 1")

activity = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_activity.csv')

# activity acc_id로 groupby
# - 평균을 내지 않은 이유 : 평균을 냈을 경우 캐릭터는 많지만
#   한 캐릭터만으로 활동한 사람의 정보가 과소평가 될 가능성이 있음
activity = activity.groupby(['acc_id'], as_index = False).sum()
activity.drop(columns = ['day','char_id'], inplace = True)
# print(activity.head())



# ======================================================================================================================


combat = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_combat.csv')

combat.drop(columns = ['day', 'server', 'char_id', 'class'], inplace = True)
combat_a = combat.groupby(['acc_id'], as_index = False).sum()
combat_a.drop(columns = 'level', inplace = True)

combat_b = combat.groupby(['acc_id'], as_index = False).max()
#  combat.groupby('acc_id', as_index = False).sum().sort_values('acc_id')

# acc_id 기준으로 정리

combat_b = combat_b[['acc_id', 'level']]
combat = pd.merge(combat_b, combat_a, how = 'inner', on = 'acc_id')



# ======================================================================================================================


payment = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_payment.csv')

payment = payment.groupby('acc_id', as_index = False).sum()
payment.drop(columns = 'day', inplace = True)

payment.rename(columns = {'amount_spent' : 'amount_spent_pay'}, inplace = True)


# ======================================================================================================================


trade = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_trade.csv')

# 거래에 참여한 횟수를 기준으로

# 판매자로서 활동한 acc_id
trade_a = trade.groupby('source_acc_id', as_index = False).count()
trade_a = trade_a[['source_acc_id', 'day']]

# 구매자로서 활동한 acc_id
trade_b = trade.groupby('target_acc_id', as_index = False).count()
trade_b = trade_b[['target_acc_id', 'day']]

x = trade_a['day'].sum() - trade_b['day'].sum()
# print(x) # 0

trade_a.rename(columns={'source_acc_id':'acc_id',
                        'day':'sell_item_cnt'}, inplace=True)
trade_b.rename(columns={'target_acc_id':'acc_id',
                        'day':'buy_item_cnt'}, inplace=True)

trade = pd.merge(trade_a, trade_b, how = 'outer', on = 'acc_id')

# 실제 데이터 검색
# trade[trade['source_acc_id'] == 6].count()

# 데이터 검증
# trade_a[trade_a['source_acc_id'] == 6]


# ======================================================================================================================


pledge = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/train_pledge.csv')

ple_1 = pledge.groupby(['server', 'pledge_id', 'day'], as_index = False).mean()
ple_1.drop(columns = ['acc_id', 'char_id', 'day'], inplace = True)
ple_1 = ple_1.groupby(['server', 'pledge_id'], as_index = False).sum()

ple_a = pledge.groupby(['acc_id', 'char_id', 'server', 'pledge_id', 'day'], as_index = False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'server', 'pledge_id'], as_index = False).sum()
ple_a = ple_a[['acc_id', 'char_id', 'server', 'pledge_id']]

pledge = pd.merge(ple_a, ple_1, how = 'outer', on = ['server', 'pledge_id'])
pledge.drop(columns = ['char_id', 'server', 'pledge_id'], inplace = True)
pledge_total = pledge.groupby(['acc_id'], as_index = False).mean()

pledge_total.rename(columns = {'random_attacker_cnt' : 'random_attacker_cnt_plg',
                              'random_defender_cnt' : 'random_defender_cnt_plg',
                              'same_pledge_cnt' : 'same_pledge_cnt_plg',
                              'temp_cnt' : 'temp_cnt_plg',
                              'etc_cnt':'etc_cnt_plg'}, inplace = True)

# ple_1[(ple_1['pledge_id'] == 25467) & (ple_1['server'] == 'aq')]
# ple_a[(ple_a['pledge_id'] == 25467) & (ple_a['server'] == 'aq')]


# ======================================================================================================================
print("process - 2")

# label + activity
label_a = pd.merge(label, activity, how = 'outer', on = 'acc_id')

# (label + activity) + combat
label_b = pd.merge(label_a, combat, how = 'outer', on = 'acc_id')

# (label + activity + combat) + payment
label_c = pd.merge(label_b, payment, how = 'outer', on = 'acc_id')

# (label + activity + combat + payment) + trade
label_d = pd.merge(label_c, trade, how = 'outer', on = 'acc_id')
label_d = label_d[label_d['survival_time'] >= 1]

# (label + activity + combat + payment + trade) + pledge_total
label_z = pd.merge(label_d, pledge_total, how = 'outer', on = 'acc_id')

data = label_z.fillna(0)
print(data.columns)

data[['playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']] = \
    scaler.fit_transform(data[['playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']])

data = data.fillna(0)
print(data.columns)


data.to_csv('AllMergeFromDonny.csv')
# ======================================================================================================================


process - 1
process - 2
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', '

# 데이터 불러오기

In [32]:
# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp
# ======================================================================================================================
scaler = MinMaxScaler()
random.seed(54654)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False
# ======================================================================================================================

data = pd.read_csv('AllMergeFromDonny.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
print(data.shape)
data.columns

(40000, 37)


Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

## 데이터를 생존과 과금여부로 나눠보기

In [33]:
# 생존여부
data.loc[data['survival_time'] < 64, "survived"] = 0
data.loc[data['survival_time'] == 64, "survived"] = 1

# 과금 여부
data.loc[data['amount_spent'] == 0, "cash"] = 0
data.loc[data['amount_spent'] > 0, "cash"] = 1

# data.drop(['acc_id','survival_time','amount_spent', 'survived'], axis=1, inplace = True)

data.head()

# 나눈 것을 다시 한번 숫자 카운팅 해보기
print(data['survived'][data['survived']==0].count(),'\n',
     data['survived'][data['survived']==1].count())

print(data['cash'][data['cash']==0].count(),'\n',
     data['cash'][data['cash']==1].count())

print(data.columns)

survival_time = data[['acc_id','survival_time']]
amount_spent = data[['acc_id','amount_spent']]

data1 = data.drop(columns=['amount_spent','cash','survived','survival_time'], axis=1)
data1.shape
# ======================================================================================================================

18004 
 21996
16438 
 23562
Index(['acc_id', 'survival_time', 'amount_spent', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent', 'amount_spent_pay', 'sell_item_cnt', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time', 'survived', 'cash'],
      dtype='object')


(40000, 35)

# 생존여부로 결정트리, 랜덤포레스트

In [4]:
# ======================================================================================================================

from sklearn.model_selection import train_test_split

y = data['survived']


x = data[['acc_id','playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent',  'sell_item_cnt','amount_spent_pay', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']]


#훈련세트, 테스트세트 나누기
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

((28000, 35), (12000, 35), (28000,), (12000,))

In [5]:
## DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

decision = DecisionTreeClassifier(max_depth = 5).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['Death', 'Survived'],
                out_file = 'decisionTree1.dot',
                impurity = True,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('payment-lable.gv', view=True)

0.6988333333333333


'payment-lable.gv.pdf'

# 생존여부를 0,1로 하여 3개의 모델을 앙상블

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import VotingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC

# log_clf = LogisticRegression(random_state=42)
# rnd_clf = RandomForestClassifier(random_state=42)
# svm_clf = SVC(random_state=42)

# # voting_clf_hard = VotingClassifier(
# #     estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
# #     voting='hard')
# # voting_clf.fit(X_train, y_train)


# voting_clf_soft = VotingClassifier(
#     estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
#     voting='soft')
# voting_clf_soft.fit(train_x, train_y)

# from sklearn.metrics import accuracy_score

# for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
#     clf.fit(train_x, train_y)
#     y_pred = clf.predict(test_x)
#     print(clf.__class__.__name__, accuracy_score(test_y, y_pred))



C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# 1~64로 랜덤포레스트 예측 해보기

In [26]:
data_cash_yes = data_cash
# data_cash_yes.columns

# data_cash_yes = data[['combat_play_time','death','enchant_count','etc_cnt',
#           'etc_cnt_plg','exp_recovery','level','non_combat_play_time',
#           'num_opponent','party_exp','play_char_cnt','pledge_combat_cnt',
#           'private_shop','quest_exp','random_attacker_cnt','random_attacker_cnt_plg',
#           'random_defender_cnt','random_defender_cnt_plg','same_pledge_cnt_plg',
#           'sell_item_cnt','temp_cnt','temp_cnt_plg','survival_time']]

# [data_cash['survival_predict']==0]
y_data = [[i] for i in data_cash_yes['survival_time'].tolist() ]
y= y_data 

data_cash_yes.drop(['survival_time'], axis=1, inplace=True)

x_data = [ data_cash_yes.iloc[i,:].tolist() for i in range(len(data_cash_yes.index.values))]
X = x_data

from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

# multi_variable_linear_regression.py
# 4

In [44]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

# 학습 진행
rfr = RandomForestClassifier(n_estimators=400, max_depth = 1,random_state=0)
rfr.fit(x_train, y_train)

gbrt = GradientBoostingClassifier(n_estimators=400, max_depth=1, learning_rate=0.01, random_state=0)
gbrt.fit(x_train, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_test, y_test)))
    
    
# 예측 
rfr_y_pred = rfr.predict(X_test)
print(rfr_y_pred)
gbc_y_pred = gbrt.predict(X_test)
print(gbc_y_pred)
print(len(list(y_test)))

# 정확도 확인
print('RandomForestClassifier 정확도 :', metrics.accuracy_score(y_test, rfr_y_pred))
print('GradientBoostingClassifier 정확도 :', metrics.accuracy_score(y_test, gbc_y_pred))

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 세트 정확도: 0.232
테스트 세트 정확도: 0.207
[64. 29. 29. ... 64.  1. 64.]
[64. 64. 29. ... 64. 64. 64.]
5526
RandomForestClassifier 정확도 : 0.14748461816865727
GradientBoostingClassifier 정확도 : 0.20684039087947884


## 생존/비생존으로 랜덤포레스트

In [34]:
# ======================================================================================================================
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train_x,train_y)
# print(clf.score(train_x, train_y))
print(rfc.score(test_x, test_y))
# rfc_y_score = rfc.predict_proba(test_x)

rfc_y_score = rfc.predict(data1)
rfc_y = pd.DataFrame(rfc_y_score, columns = ['survival_predict'])
print(type(rfc_y))
rfc_y.head()
# ======================================================================================================================


0.754
<class 'pandas.core.frame.DataFrame'>


,survival_predict
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0


# 랜덤포레스트 결과에서 예측값 뽑기

In [ ]:
rfc_y_score = rfc.predict(data1)
rfc_y = pd.DataFrame(rfc_y_score, columns = ['survival_predict'])
print(type(rfc_y))

In [35]:
data_cash = pd.concat([data1, rfc_y], axis=1)
data_cash.head()
rfc_y.shape
print(len(data_cash.columns))

<class 'pandas.core.frame.DataFrame'>


In [37]:
data_cash = data_cash.merge(survival_time, on='acc_id', how='inner')
print(data_cash.columns)
data_cash.shape
print(data_cash[data_cash['survival_predict']==1].shape, data_cash[data_cash['survival_time']==64].shape)
print(data_cash[data_cash['survival_predict']==0].shape, data_cash[data_cash['survival_time']<64].shape)

Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'level',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'amount_spent_pay',
       'sell_item_cnt', 'buy_item_cnt', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_plg',
       'random_defender_cnt_plg', 'same_pledge_cnt_plg', 'temp_cnt_plg',
       'etc_cnt_plg', 'combat_play_time', 'non_combat_play_time',
       'survival_predict', 'survival_time'],
      dtype='object')
(23422, 37) (21996, 37)
(16578, 37) (18004, 37)


# 생존이 0(도중 이탈자)인 인원들로 다시 해보기

In [38]:
data_cash_yes = data_cash[data_cash[ 'survival_predict'] == 0 ]
data_cash_yes.columns



y_data = [[i] for i in data_cash_yes['survival_time'].tolist() ]
y = y_data 

data_cash_yes.drop(['acc_id','survival_time','survival_predict'], axis=1, inplace=True)

x_data = [ data_cash_yes.iloc[i,:].tolist() for i in range(len(data_cash_yes.index.values))]
X = x_data

from sklearn.model_selection import train_test_split

x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) 


# multi_variable_linear_regression.py
# 4

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
import pandas as pd
 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# 학습 진행
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)
 
# 예측 
y_pred = forest.predict(X_test)
print(y_pred)
print(list(y_test))
 
# 정확도 확인
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


[18. 29. 29. ...  1. 48. 11.]
[[10.0], [11.0], [29.0], [6.0], [8.0], [1.0], [29.0], [9.0], [54.0], [58.0], [34.0], [64.0], [22.0], [20.0], [29.0], [38.0], [1.0], [17.0], [26.0], [46.0], [17.0], [21.0], [14.0], [3.0], [29.0], [8.0], [2.0], [36.0], [4.0], [23.0], [21.0], [29.0], [10.0], [1.0], [1.0], [8.0], [17.0], [27.0], [24.0], [48.0], [2.0], [8.0], [36.0], [1.0], [48.0], [7.0], [10.0], [28.0], [29.0], [4.0], [29.0], [36.0], [1.0], [22.0], [60.0], [8.0], [19.0], [8.0], [6.0], [14.0], [33.0], [29.0], [32.0], [11.0], [18.0], [1.0], [14.0], [29.0], [1.0], [4.0], [20.0], [10.0], [29.0], [1.0], [3.0], [13.0], [52.0], [2.0], [29.0], [2.0], [4.0], [26.0], [52.0], [1.0], [22.0], [64.0], [8.0], [26.0], [64.0], [11.0], [37.0], [10.0], [1.0], [8.0], [31.0], [2.0], [10.0], [8.0], [18.0], [7.0], [5.0], [13.0], [1.0], [25.0], [21.0], [46.0], [29.0], [14.0], [1.0], [29.0], [10.0], [20.0], [3.0], [8.0], [1.0], [15.0], [13.0], [10.0], [64.0], [20.0], [10.0], [60.0], [58.0], [22.0], [29.0], [4.0], [10.

# 과금여부로 결정트리, 랜덤포레스트

In [23]:
import tensorflow as tf


X = tf.placeholder(tf.float32,shape=[None,data_cash_yes.shape[1]])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([data_cash_yes.shape[1],1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W) + b  #

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=3e-1)
train = optimizer.minimize(cost)



# RMSE( root-mean-square error ) : 평균 제곱 근 오차
# 평균 제곱근편차(RMSD) 또는 평균 제곱근오차(RMSE)는 추정값 또는 모델이 예측한 값과
# 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다.
# 정밀도를 표현하는데 적합하다.

targets = tf.placeholder(tf.float32,[None,1])  # 답(Y)
predictions = tf.placeholder(tf.float32,[None,1]) # 예측 값
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# start training
for step in range(10001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step, cost_val, W_val, b_val)

# Test step
test_predict = sess.run(hypothesis,feed_dict={X:X_test})
rmse_val = sess.run(rmse,feed_dict={targets:y_test,predictions:test_predict})
print('RMSE: {}'.format(rmse_val))


total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, test_predict)))
R_squared = tf.subtract(1.0,tf.divide( unexplained_error,total_error))

print('R_sqr: {}'.format(sess.run(R_squared)))

prd = sess.run(hypothesis, feed_dict = {X:X_test})

<class 'pandas.core.frame.DataFrame'>
36


Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'level',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'amount_spent_pay',
       'sell_item_cnt', 'buy_item_cnt', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_plg',
       'random_defender_cnt_plg', 'same_pledge_cnt_plg', 'temp_cnt_plg',
       'etc_cnt_plg', 'combat_play_time', 'non_combat_play_time',
       'cash_predict'],
      dtype='object')

# 랜덤포레스트로 회귀 분석해보기

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 과금 여부
data.loc[data['amount_spent'] == 0, "cash"] = 0
data.loc[data['amount_spent'] > 0, "cash"] = 1

print(data['cash'][data['cash']==0].count(),'\n',
     data['cash'][data['cash']==1].count())

amount_spent = data[['acc_id','amount_spent']]

data1 = data.drop(columns=['amount_spent','cash','survived','survival_time'], axis=1)
data1.shape

# ==========================================================================================================

y = data['cash']


x = data[['acc_id','playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count', 'level', 'pledge_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'temp_cnt', 'same_pledge_cnt', 'etc_cnt',
       'num_opponent',  'sell_item_cnt','amount_spent_pay', 'buy_item_cnt',
       'play_char_cnt', 'combat_char_cnt', 'pledge_combat_cnt',
       'random_attacker_cnt_plg', 'random_defender_cnt_plg',
       'same_pledge_cnt_plg', 'temp_cnt_plg', 'etc_cnt_plg',
       'combat_play_time', 'non_combat_play_time']]


#훈련세트, 테스트세트 나누기
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

16438 
 23562


((28000, 35), (12000, 35), (28000,), (12000,))

In [54]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train_x, train_y)
rfc_y_score = rfc.predict(data1)
rfc_y = pd.DataFrame(rfc_y_score, columns = ['amount_predict'])
print(type(rfc_y))
rfc.score(test_x,test_y)

<class 'pandas.core.frame.DataFrame'>


0.8574166666666667

In [55]:
data_cash = pd.concat([data1, rfc_y], axis=1)
data_cash.head()
rfc_y.shape
print(len(data_cash.columns))

data_cash = data_cash.merge(survival_time, on='acc_id', how='inner')
print(data_cash.columns)
data_cash.shape
print(data_cash[data_cash['amount_predict']==1].shape, data_cash[data_cash['amount_predict']==64].shape)
print(data_cash[data_cash['amount_predict']==0].shape, data_cash[data_cash['amount_predict']<64].shape)

36
Index(['acc_id', 'playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'level',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'amount_spent_pay',
       'sell_item_cnt', 'buy_item_cnt', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_plg',
       'random_defender_cnt_plg', 'same_pledge_cnt_plg', 'temp_cnt_plg',
       'etc_cnt_plg', 'combat_play_time', 'non_combat_play_time',
       'amount_predict', 'survival_time'],
      dtype='object')
(24623, 37) (0, 37)
(15377, 37) (40000, 37)


In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

data_cash = data1.merge(amount_spent, on='acc_id', how='inner')

y_data = [[i] for i in data_cash['amount_spent'].tolist() ]
y= y_data 

data_cash.drop(['amount_spent'], axis=1, inplace=True)

x_data = [ data_cash.iloc[i,:].tolist() for i in range(len(data_cash.index.values))]
X = x_data



x_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

# multi_variable_linear_regression.py
# 4

In [48]:
rfr = RandomForestRegressor(n_estimators=1000,max_depth=None, criterion = 'mse', random_state=1, n_jobs = -1)

rfr.fit(x_train, y_train)
y_train_pred = rfr.predict(x_train)
y_test_pred = rfr.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print('R2 score \n Train : {0:.3f}, \t Test : {1:.3f}'.format(round(r2_train,3), round(r2_test,3)))

# sort_value = data_cash.ravel().argsort()
# plt.plot(data_cash[sort_value], rfr.predict(data_cash[sort_value]), c='r', lw=1, label='랜덤포레스트' )

# plt.scatter(x_train, y_train, c='lightgray', marker='o', label = '트레이닝 데이터')
# plt.scatter(x_test, y_test, c='greem', marker='s', label = '테스트 데이터')
# plt.legend()
# plt.show()

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


R2 score 
 Train : 0.861, 	 Test : 0.051


In [47]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

NameError: name 'log_clf' is not defined